# Navigator Real Time Inference

This Notebook introduces Gretel Navigator's real-time inference. Gretel Navigator enables generation or editing of tabular data from a user-provided prompt. In the examples below we will explore:

* Generating a table from a prompt
* Augmenting an existing table from a prompt
* Streaming records one-by-one

For more details on Gretel Navigator please see Gretel's documentation:

https://docs.gretel.ai/create-synthetic-data/models/navigator/getting-started

In [ ]:
%%capture
!pip install -U "gretel-client>=0.18"

In [ ]:
# Sign-up and get an API key from: https://console.gretel.ai/users/me/key

from gretel_client import Gretel

gretel = Gretel(api_key="prompt")

In [ ]:
# NOTE: Please see our docs FAQ regarding the different models you can choose from
# https://docs.gretel.ai/create-synthetic-data/models/navigator/faq
GRETEL_MODEL = "gretelai/tabular-v0"

navigator = gretel.factories.initialize_inference_api("navigator", backend_model=GRETEL_MODEL)

# Generate a table from a prompt

Below we provide a prompt that outlines what kind of table to generate as well
as the columns that are desired to be included.

In [ ]:
PROMPT = """
Generate a dataset of characters from The Simpsons.

Each character should have the following columns:
* first_name: The first name of the character.
* last_name: The last name of the character.
* favorite_band: The character's all-time favorite band.
* favorite_tv_show: The character's favorite TV show other than The Simpsons.
* favorite_food: The character's favorite food.
* backstory: The character's back story in 3-5 sentences.
"""

In [ ]:
df = navigator.generate(prompt=PROMPT, num_records=10)

df

# Edit a table from a prompt

Now, given an existing table, we can also ask Navigator to augment this table with additional data.

In [ ]:
EDIT_PROMPT = """
Add exactly the following columns to the provided table:

* worst_nightmare: Describe the character's worst nightmare
"""

In [ ]:
df = navigator.edit(prompt=EDIT_PROMPT, seed_data=df)

df

# Streaming records

When generating data, Navigator generates data in batches with a maxium of 50 records per batch. Whenever `num_records` has a value greater than 50, the SDK will automatically request new batches until the target number is reached.

When generating records above 50, sometimes it's helpful to pass some of the records from a previous batch into the next batch's generation request. This can help retain context between batches.

Additionally, instead of waiting for an entire table to be generated, the SDK can be used with a `streaming=True` option which converts the generation method to a Python generator and you can process records as they arrive from Navigator.


In [ ]:
PROMPT = """
Generate positive and negative reviews for common household products purchased online.

Columns are: the product name, number of stars (1-5), review and customer id
"""

for record in navigator.generate(
    prompt=PROMPT,
    num_records=60,
    stream=True,
    sample_buffer_size=10
):
    print(record)